In [1]:
import numpy as np
import mir_eval.transcription
import os
from IPython.display import display, Markdown

import transcribe_factorization as tf
import evaluate_transcription as et
import STFT
import pandas as pd

In [2]:
def compute_scores_database_one_condition(piano_type, T, note_intensity, H_normalization = False):
    #path_computed_H = "C:\\Users\\amarmore\\Desktop\\cnmf_res\\activations"
    path_computed_H = "../data_persisted/activations"
    #path_songs = "C:\\Users\\amarmore\\Desktop\\Audio samples\\MAPS\\{}\\MUS".format(piano_type)
    path_songs = "/home/jecohen/Travail/Toolbox/Data/Audio_Prozik/Maps/MAPS/{}/MUS".format(piano_type)
    
    time_limit = 30
    beta = 1
    init = "L1"
    model_AD = True
    itmax_H = 100
    tol = 1e-8
    codebook = range(21, 109)
    onset_tolerance = 150/1000
    
    #a = np.arange(1e-3, 1e-2, 1e-3)
    #b = np.arange(1e-4, 1e-3, 1e-4)
    #c = np.arange(1e-5, 1e-4, 1e-5)
    #d = np.arange(1e-6, 1e-5, 1e-6)
    f = np.arange(1e-2, 2e-1, 1e-2)
    g = np.arange(0.2, 0.5, 0.1)
    #h = np.arange(1, 1.5, 0.1)
    #j = np.arange(1e-7, 1e-6, 1e-7)
    #listthres = np.r_[h[::-1], g[::-1], f[::-1], a[::-1], b[::-1], c[::-1], d[::-1], j[::-1]]
    listthres = np.r_[g[::-1], f[::-1]]

    files = os.listdir(path_songs)
    list_files_wav = []
    for it_files in files:
        if it_files.split(".")[-1] == "wav":
            list_files_wav.append(it_files)
            
    all_res = []
    for a_song in list_files_wav:
        song_name = a_song.replace(".wav", "")
        print("processing piano song: {}".format(song_name))
        path_this_song = "{}/{}".format(path_songs, a_song)
        stft = STFT.STFT(path_this_song, time = time_limit, model_AD=model_AD, channel = 0)

        X = stft.get_magnitude_spectrogram()
        H_persisted_name = "activations_song_{}_W_learned_{}_beta_{}_T_{}_init_{}_stftAD_{}_itmax_{}_intensity_W_{}_time_limit_{}_tol_{}".format(song_name, piano_type, beta, T, init, model_AD, itmax_H, note_intensity, time_limit, tol)

        annot_name = a_song.replace("wav","txt")
        annot_this_song = "{}/{}".format(path_songs, annot_name)
        note_annotations = et.load_ref_in_array(annot_this_song, time_limit=time_limit)
        ref = np.array(note_annotations, float)
        ref_pitches = np.array(ref[:,2], int)
        try:
            H = np.load("{}/{}.npy".format(path_computed_H, H_persisted_name), allow_pickle = True)
            res_each_song = []
            for threshold in listthres:
                prediction, _ = tf.transcribe_activations(codebook, H, stft, threshold, H_normalization = H_normalization)
                est = np.array(prediction, float)
                if est.size > 0:
                    est_pitches = np.array(est[:,2], int)
                    (prec, rec, f_mes, _) = mir_eval.transcription.precision_recall_f1_overlap(ref[:,0:2], ref_pitches, est[:,0:2], est_pitches, offset_ratio = None, onset_tolerance = onset_tolerance)
                    matching = mir_eval.transcription.match_notes(ref[:,0:2], ref_pitches, est[:,0:2],est_pitches, onset_tolerance=onset_tolerance,offset_ratio=None)
                    TP = len(matching)
                    try:
                        FP = int(TP * (1 - prec) / prec)
                    except ZeroDivisionError:
                        FP = 0
                    try:
                        FN = int(TP * (1 - rec) / rec)
                    except ZeroDivisionError:
                        FN = 0
                    acc = et.accuracy(TP,FP,FN)
                else:
                    prec, rec, f_mes, acc, TP, FP, FN = (0,0,0,0,0,0,0)
                #TP_temp, FP_temp, FN_temp = et.compute_statistical_rates_on_array(note_annotations, prediction, onset_tolerance=onset_tolerance)
                res_each_song.append([prec, rec, f_mes, acc, TP, FP, FN])

            all_res.append(res_each_song)

        except FileNotFoundError:
            print("\033[91m This song failed: {} \033[00m".format(a_song)) 
            pass
        
    np_all_res = np.array(all_res)
    lines = []
    col = ['Precision', 'Recall', 'F measure','Accuracy','True Positives','False Positives','False Negatives']
    for idx, threshold in enumerate(listthres):
        lines.append([round(np.mean(np_all_res[:,idx,i]), 4) for i in range(len(col))])
    df = pd.DataFrame(np.array(lines), columns = col, index = np.round(listthres,decimals = 5))
    display(df.style.bar(subset=["F measure", "Accuracy"], color='#5fba7d'))
    
    best_val = []
    for each_song in range(len(list_files_wav)):
        best_thresh_idx = np.argmax(np_all_res[each_song,:,2])
        best_val.append([round(np_all_res[each_song,best_thresh_idx,i], 4) for i in range(len(col))])
    
    best_df = pd.DataFrame(np.array([np.mean(np.array(best_val)[:,i]) for i in range(len(col))]), columns = ["best res when optimizing thresh"], index = col)
    display(best_df.T)

# Sans normalisation des H

In [5]:
#print('piano_type = "AkPnCGdD", T = 10, note_intensity = "F"')
#compute_scores_database_one_condition(piano_type = "AkPnCGdD", T = 10, note_intensity = "F", H_normalization = False)

#print('piano_type = "AkPnCGdD", T = 10, note_intensity = "M"')
#compute_scores_database_one_condition(piano_type = "AkPnCGdD", T = 10, note_intensity = "M", H_normalization = False)

#print('piano_type = "AkPnCGdD", T = 10, note_intensity = "P"')
#compute_scores_database_one_condition(piano_type = "AkPnCGdD", T = 10, note_intensity = "P", H_normalization = False)


#print('\n piano_type = "ENSTDkCl", T = 10, note_intensity = "F"')
#compute_scores_database_one_condition(piano_type = "ENSTDkCl", T = 10, note_intensity = "F", H_normalization = False)

#print('piano_type = "ENSTDkCl", T = 10, note_intensity = "M"')
#compute_scores_database_one_condition(piano_type = "ENSTDkCl", T = 10, note_intensity = "M", H_normalization = False)

#print('piano_type = "ENSTDkCl", T = 10, note_intensity = "P"')
#compute_scores_database_one_condition(piano_type = "ENSTDkCl", T = 10, note_intensity = "P", H_normalization = False)

#print('piano_type = "SptkBGAm", T = 10, note_intensity = "M"')
#compute_scores_database_one_condition(piano_type = "SptkBGAm", T = 10, note_intensity = "M", H_normalization = False)

print('piano_type = "StbgTGd2", T = 10, note_intensity = "M"')
compute_scores_database_one_condition(piano_type = "StbgTGd2", T = 10, note_intensity = "M", H_normalization = False)


piano_type = "StbgTGd2", T = 10, note_intensity = "M"
processing piano song: MAPS_MUS-mz_570_1_StbgTGd2
processing piano song: MAPS_MUS-schuim-1_StbgTGd2
processing piano song: MAPS_MUS-mz_311_1_StbgTGd2
processing piano song: MAPS_MUS-bk_xmas1_StbgTGd2
processing piano song: MAPS_MUS-mz_331_3_StbgTGd2
processing piano song: MAPS_MUS-liz_rhap09_StbgTGd2
processing piano song: MAPS_MUS-bor_ps6_StbgTGd2
processing piano song: MAPS_MUS-grieg_butterfly_StbgTGd2
processing piano song: MAPS_MUS-mz_333_2_StbgTGd2
processing piano song: MAPS_MUS-scn15_11_StbgTGd2
processing piano song: MAPS_MUS-scn16_3_StbgTGd2
processing piano song: MAPS_MUS-mz_545_3_StbgTGd2
processing piano song: MAPS_MUS-deb_menu_StbgTGd2
processing piano song: MAPS_MUS-pathetique_1_StbgTGd2
processing piano song: MAPS_MUS-liz_et6_StbgTGd2
processing piano song: MAPS_MUS-ty_mai_StbgTGd2
processing piano song: MAPS_MUS-ty_maerz_StbgTGd2
processing piano song: MAPS_MUS-bk_xmas5_StbgTGd2
processing piano song: MAPS_MUS-mz_331

,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
0.4,0.497700,0.038000,0.064600,0.037900,9.466700,0.300000,118.633300
0.3,0.723600,0.069600,0.112800,0.068600,18.133300,0.966700,166.500000
0.2,0.919300,0.187600,0.278200,0.179100,48.300000,5.333300,194.266700
0.19,0.917200,0.213800,0.311500,0.204000,54.700000,6.266700,187.700000
0.18,0.914000,0.243600,0.347700,0.232200,61.900000,6.933300,180.600000
0.17,0.914700,0.278300,0.389500,0.264700,69.866700,7.966700,172.633300
0.16,0.915700,0.314200,0.430000,0.299000,78.366700,8.433300,164.100000
0.15,0.909700,0.347800,0.466300,0.330200,86.500000,9.100000,156.100000
0.14,0.905600,0.378700,0.497100,0.358600,94.433300,10.733300,148.033300
0.13,0.901500,0.411200,0.529100,0.388400,102.866700,11.733300,139.600000


,Precision,Recall,F measure,Accuracy,True Positives,False Positives,False Negatives
best res when optimizing thresh,0.8236,0.845497,0.83136,0.720613,210.566667,42.933333,34.266667


# Avec normalisation des H

In [4]:
print('piano_type = "AkPnCGdD", T = 10, note_intensity = "F"')
compute_scores_database_one_condition(piano_type = "AkPnCGdD", T = 10, note_intensity = "F", H_normalization = True)

print('piano_type = "AkPnCGdD", T = 10, note_intensity = "M"')
compute_scores_database_one_condition(piano_type = "AkPnCGdD", T = 10, note_intensity = "M", H_normalization = True)

print('piano_type = "AkPnCGdD", T = 10, note_intensity = "P"')
compute_scores_database_one_condition(piano_type = "AkPnCGdD", T = 10, note_intensity = "P", H_normalization = True)


print('\n piano_type = "ENSTDkCl", T = 10, note_intensity = "F"')
compute_scores_database_one_condition(piano_type = "ENSTDkCl", T = 10, note_intensity = "F", H_normalization = True)

print('piano_type = "ENSTDkCl", T = 10, note_intensity = "M"')
compute_scores_database_one_condition(piano_type = "ENSTDkCl", T = 10, note_intensity = "M", H_normalization = True)

print('piano_type = "ENSTDkCl", T = 10, note_intensity = "P"')
compute_scores_database_one_condition(piano_type = "ENSTDkCl", T = 10, note_intensity = "P", H_normalization = True)

piano_type = "AkPnCGdD", T = 10, note_intensity = "F"
processing piano song: MAPS_MUS-liz_et_trans5_AkPnCGdD
 This song failed: MAPS_MUS-liz_et_trans5_AkPnCGdD.wav 
processing piano song: MAPS_MUS-muss_2_AkPnCGdD
 This song failed: MAPS_MUS-muss_2_AkPnCGdD.wav 
processing piano song: MAPS_MUS-ty_februar_AkPnCGdD
 This song failed: MAPS_MUS-ty_februar_AkPnCGdD.wav 
processing piano song: MAPS_MUS-chpn_op27_1_AkPnCGdD
 This song failed: MAPS_MUS-chpn_op27_1_AkPnCGdD.wav 
processing piano song: MAPS_MUS-mendel_op53_5_AkPnCGdD
 This song failed: MAPS_MUS-mendel_op53_5_AkPnCGdD.wav 
processing piano song: MAPS_MUS-alb_esp3_AkPnCGdD
 This song failed: MAPS_MUS-alb_esp3_AkPnCGdD.wav 
processing piano song: MAPS_MUS-grieg_kobold_AkPnCGdD
 This song failed: MAPS_MUS-grieg_kobold_AkPnCGdD.wav 
processing piano song: MAPS_MUS-chpn-p20_AkPnCGdD
 This song failed: MAPS_MUS-chpn-p20_AkPnCGdD.wav 
processing piano song: MAPS_MUS-chpn-p11_AkPnCGdD
 This song failed: MAPS_MUS-chpn-p11_AkPnCGdD.wav 
pro

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed